### Various scenarios for the Zenodo Client Constructor

#### No/default/None arguments (where token=None and config_path=None or "")

In this scenario, the constructor is called without any user-defined arguments. There are two possible responses: 1) a config file already exists in `~/.zenodorc`; 2) The `~/.zenodorc` does not exist. 

The first case is trivial as the existing config file is read and used by the `configparser` library in the `Zenodo` class in `client` module. The `configparser` library raises a `MissingSectionHeaderError` exception if the existing config file does have a section header such as **[ZENODO]** or **[SANDBOX]**. 

In the second scenario where a `.zenodorc` file does not exist in Home directory, Zenodo class constructor creates a `.zenodorc` file in Home directory that includes two sections: **[ZENODO]** and **[SANDBOX]**. Depending on the value of the `use_sandbox` boolean argument, the corresponding section will be populated with an empty `token` that has to he initialized by the user. Simultaneously, the user will be notified with a warning message about these actions.

In [1]:
import client

In [2]:
zen = client.Zenodo(use_sandbox=False)

List the existing sections in the config object:

In [3]:
zen.list_sections()

['ZENODO', 'SANDBOX']

Let us also make sure that the empty `token` placeholder is created in the corresponding section of the config object, assuming the `use_sandbox = False`.

In [3]:
zen.list_tokens("ZENODO")

[('token', 'some_value'), ('new_token', 'new_value')]

If an empty config file address or empty string is provided for the `config_file_path` argument as `config_file_path=""` and the `token` argument is `None`, the constructor will also interpret the `config_file_path`'s value as `None` and behave similar to the scenario mentioned above.

In [5]:
zen = client.Zenodo(config_file_path="", use_sandbox=False)

Once the empty `.zenodorc` is created, the empty `token` can be populated either manually or using thje `write_token()` `Zenodo` class member function.

In [6]:
zen.write_token("Zenodo", "new_token", "new_value", force_rewrite=True)

In [7]:
zen.list_tokens("Zenodo")

[('token', 'some_value'), ('new_token', 'new_value')]

Note that although our convention is to use all capitalized letters for the section headers (*e.g.*, **ZENODO**) when working with config files, all `Zenodo` class member functions are agnostic to the capitalization.

Once the token is stored in `Zenodo`'s instance's file object, the new changes can be committed/saved into the config file on disk using `update_config_file()`

In [8]:
zen.update_config_file()

Let us check to see if the changes were applied correctly

In [15]:
for sec in zen.list_sections():
    print("[%s]:" % sec)
    for key, value in zen.list_tokens(sec):
        print("\t %s = %s" % (key, value))
    print()

[ZENODO]:
	 token = some_value
	 new_token = new_value

[SANDBOX]:



Trying to create a new config file in a location that already exists raises and exception

In [4]:
zen.create_config_file('~/.zenodorc')

Error: A config file already exists in '/home/sina/.zenodorc'.

In [8]:
zen.read_token("ZENODO", "token")

'some_value'

In [16]:
import configparser
from pathlib import Path

In [17]:
path = "~/.zenodorc"
# path=""
path = Path(path).expanduser()
config = configparser.ConfigParser()
config.read(path)
section = "ZENODO"
key = 'token'
# print(dict((key, list(value)) for key, value in config.items()))
# config.default_section = "MOLSSI"
# config.default_section
# config.has_section("ZENODO")
# config.has_option("ZENODO", key)
# config.get("NO", key)
config.sections()
# path

['ZENODO', 'SANDBOX']

In [ ]:
import json
str(json.dumps(dict(config["ZENODO"].items()), indent=4))

In [18]:
config